In [1]:
# !pip install accelerate -U
!pip install transformers[torch]

In [2]:

!pip install transformers

In [3]:
#  Python 프로그램에서 데이터셋을 쉽게 다룰 수 있게 해주는 datasets 라이브러리를 설치하는 작업
!pip install datasets

In [4]:

!pip install evaluate

In [5]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer # baseline code용 tfidef vectorirzer(또는 tokenizer)
from sklearn import preprocessing
from sklearn.metrics import f1_score
from IPython.display import display, HTML

from transformers import AutoModel, AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer # 사용하고자 하는 모델, 토크나이저 적용시 필요
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, ClassLabel

from tqdm.auto import tqdm # process bar 표시용

import warnings
warnings.filterwarnings(action='ignore')

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device  # pt 코드에서 필요함

device(type='cuda')

In [7]:
# Hugging Face의 datasets 라이브러리를 사용하여 "klue"의 "ynat" 데이터셋을 불러와 datasets라는 변수에 저장
datasets = load_dataset("klue", "ynat") # "klue" 데이터셋에서 "ynat" 부분집합을 불러오기

In [8]:
# 데이터셋 내에 어떻게 들어있나 보기 위한 함수

# 임의의 예제를 선택하여 출력하는 함수를 정의
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []

    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

show_random_elements(datasets["validation"])

,guid,title,label,url,date
0,ynat-v1_dev_03289,화천산천어축제 밤낚시 이용객 70% 화천서 숙박,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=242&oid=001&aid=0009000167,2017.02.01. 오후 3:42
1,ynat-v1_dev_00426,JTBC 뉴스룸 주말 김필규한민용 앵커 체제,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0010242829,2018.07.31. 오전 10:32
2,ynat-v1_dev_01295,김정일 통역사 출신 김성남 北 국제부 부부장 방중,정치,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=231&oid=001&aid=0008675661,2016.09.09. 오전 9:27
3,ynat-v1_dev_04104,조각가 한애규가 따뜻한 흙으로 빚은 푸른 길,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=242&oid=001&aid=0010156830,2018.06.18. 오후 5:44
4,ynat-v1_dev_08957,박인아 16점…한국 U18 아시아 여자농구서 대만 제압,스포츠,https://sports.news.naver.com/news.nhn?oid=001&aid=0010430831,2018.10.29 06:35
5,ynat-v1_dev_06857,민주 부산시당 공천관리위 구성…위원장 황호선 전교수,정치,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=001&aid=0009941697,2018.03.07. 오후 3:27
6,ynat-v1_dev_08451,롯데 손용석 코치 3일 KIA전서 아버지와 시구·시포,스포츠,https://sports.news.naver.com/news.nhn?oid=001&aid=0010059826,2018.05.01 15:49
7,ynat-v1_dev_05434,가짜 서류로 경력 속여 소방공무원 응시 합격,사회,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=257&oid=001&aid=0011992520,2020.11.04. 오후 3:33
8,ynat-v1_dev_01814,증시풍향계 中 우한 폐렴 추이와 美 FOMC 결과 주목,세계,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=258&oid=001&aid=0011356681,2020.01.25. 오전 9:02
9,ynat-v1_dev_04199,사진 기획전 언노운 노운·이와이 도시오 특별전,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=242&oid=001&aid=0009107043,2017.03.14. 오후 5:12


In [9]:
pretrained_model = 'monologg/kobigbird-bert-base'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [10]:
test_model = AutoModel.from_pretrained(pretrained_model)
test_model

BigBirdModel(
  (embeddings): BigBirdEmbeddings(
    (word_embeddings): Embedding(32500, 768, padding_idx=0)
    (position_embeddings): Embedding(4096, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BigBirdEncoder(
    (layer): ModuleList(
      (0-11): 12 x BigBirdLayer(
        (attention): BigBirdAttention(
          (self): BigBirdBlockSparseAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): BigBirdSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
    

In [11]:
sentences = ["안녕하세요 hmkd 여러분", "무더운 여름에 건강관리 잘하세요!"]
features = tokenizer(
    sentences,
    max_length=10,
    padding="max_length",
    truncation=True,
)
features

{'input_ids': [[2, 9279, 4681, 9421, 576, 4522, 4507, 4565, 8280, 3], [2, 26787, 7775, 4584, 7206, 11066, 3747, 4681, 9421, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [12]:
features = {k:torch.tensor(v) for k,v in features.items()}
output_tensor=test_model(**features)
print(output_tensor)

Attention type 'block_sparse' is not possible if sequence_length: 10 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0085,  0.3430, -0.4673,  ...,  0.1736, -0.2570,  0.0558],
         [ 0.4125, -0.6818, -0.1634,  ...,  0.0846,  0.1364,  0.4318],
         [ 0.4620,  0.0325,  0.0777,  ...,  0.1734,  0.0707,  0.1572],
         ...,
         [-0.2735,  0.2188,  0.1350,  ..., -0.2034,  0.2622,  0.4731],
         [ 0.1283,  0.2342, -0.3019,  ..., -0.2352,  0.1830,  0.6579],
         [-0.0145,  0.0533,  0.2154,  ..., -0.1981, -0.0600,  0.3875]],

        [[ 0.3253, -0.0567, -0.1669,  ...,  0.0151,  0.3085,  0.1041],
         [ 0.0719, -0.0903,  0.0267,  ..., -0.2625,  0.7100,  0.1628],
         [-0.0882, -0.0213, -0.1715,  ..., -0.1045,  0.8717,  0.2090],
         ...,
         [ 0.1878, -0.0069, -0.2244,  ...,  0.2413,  0.4385, -0.0058],
         [ 0.1836, -0.0750,  0.2445,  ...,  0.2398,  0.3535, -0.1645],
         [-0.1313, -0.2876, -0.0321,  ..., -0.2087,  0.4649,  0.2791]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [13]:
# output_tensor가 텐서가 아니라 사전 또는 다른 구조인 경우(복잡한 모델을 처리할 때 자주 발생할 수 있음) 관심 있는 특정 텐서에 먼저 액세스해야 함
for key, value in output_tensor.items():
    print(f"Size of tensor '{key}': {value.size()}")

Size of tensor 'last_hidden_state': torch.Size([2, 10, 768])
Size of tensor 'pooler_output': torch.Size([2, 768])


In [14]:
# 데이터 tokenize
def preprocess_function(examples):
    return tokenizer(examples["title"], truncation=True,  return_token_type_ids=False,) # 토큰의 타입 정보를 반환하지 않도록 함
# preprocess_function은 데이터셋의 각 예제에 대해 토큰화 작업을 수행하고, 토큰화된 결과를 새로운 데이터셋에 저장. batched=True로 설정되어 있으므로, preprocess_function은 여러 예제를 동시에 처리
tokenized_datasets = datasets.map(preprocess_function, batched=True)

In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date', 'input_ids', 'attention_mask'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date', 'input_ids', 'attention_mask'],
        num_rows: 9107
    })
})

In [16]:
# loss type define
import evaluate
accuracy = evaluate.load("accuracy")

In [17]:
# 기사의 본문을 입력으로 받아서 해당 기사가 7개 topic중 어느 것에 속하는지를 분류
# loss(accuracy) 적용
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "IT과학", 1: "경제", 2: "사회", 3: "생활문화", 4: "세계", 5:"스포츠", 6: "정치" }
label2id = {"IT과학":0, "경제":1, "사회":2, "생활문화":3, "세계":4, "스포츠":5, "정치":6 }

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=7, id2label=id2label, label2id=label2id)

Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model

BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [20]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
!pip install transformers[torch]

In [22]:
!pip install accelerate -U

In [23]:
training_args = TrainingArguments(
    output_dir="topic_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Attention type 'block_sparse' is not possible if sequence_length: 18 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.379000,0.384810,0.869880


TrainOutput(global_step=2855, training_loss=0.4653413548778527, metrics={'train_runtime': 339.5556, 'train_samples_per_second': 134.523, 'train_steps_per_second': 8.408, 'total_flos': 495932362009212.0, 'train_loss': 0.4653413548778527, 'epoch': 1.0})

In [25]:
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy
0,0.342900,0.402024,0.866367


{'eval_loss': 0.40202370285987854, 'eval_accuracy': 0.8663665312397058}

In [26]:
model.save_pretrained('topic_model')

In [30]:
text ='국회, 2월 임시회 개회…6~8일 대정부질문 여야 주자는?'
inputs =  tokenizer(text, truncation=True,  return_token_type_ids=False, return_tensors="pt").to(device)
model.eval() # 모델을 평가 모드로 설정
outputs = model(**inputs)
predictions = outputs.logits.argmax(-1) # argmax(-1)는 마지막 차원(일반적으로 클래스 차원)을 따라 가장 큰 값의 인덱스를 반환
id2label[predictions.cpu().numpy()[0]] # 예측된 클래스 인덱스를 넘파이 배열로 변환하고 그 주 첫번째 값을 레이블로 변경

'정치'

In [31]:
!mkdir /content/drive/MyDrive/nlpbook/best_model

mkdir: cannot create directory ‘/content/drive/MyDrive/nlpbook/best_model’: File exists


In [32]:
!ls

'=0.20.1'   drive   sample_data   topic_model


In [34]:
!cp -r ./topic_model /content/drive/MyDrive/nlpbook/best_model

In [35]:
# 한글 깨짐 해결방법 : 실행 후 Restart runtime
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (4,165 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120500 files and dire

In [36]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [37]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device  # pt 코드에서 필요함

device(type='cuda')

In [38]:
pretrained_model = 'monologg/kobigbird-bert-base'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [40]:
# 기사의 본문을 입력으로 받아서 해당 기사가 7개 topic중 어느 것에 속하는지를 분류
# loss(accuracy) 적용
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "IT과학", 1: "경제", 2: "사회", 3: "생활문화", 4: "세계", 5:"스포츠", 6: "정치" }
label2id = {"IT과학":0, "경제":1, "사회":2, "생활문화":3, "세계":4, "스포츠":5, "정치":6 }

In [41]:
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/nlpbook/best_model/topic_model')

In [42]:
model

BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [52]:
def inference_fn(sentence) :
  inputs = tokenizer(sentence, truncation = True, return_token_type_ids = False, return_tensors = 'pt').to(device)
  model.eval()
  outputs = model(**inputs)
  predictions = outputs.logits.argsmax(-1)

  return {
      'sentence' : sentence,
      'prediction' : id2label[predictions.cpu().numpy([0])]
  }

In [53]:
sentence = '대통령이 국정연설을 한다.'
inference_fn(sentence)

RuntimeError: ignored

In [ ]:
!mkdir  /root/.ngrok2 && echo 'authtoken: 2T5dj5MGKLVe9gdeTmRn96rqUGK_6bLTw98KhQnQENd1C9FCt' > /root/.ngrok2/ngrok.yml

In [ ]:
!pipi install flask_ngrok

In [ ]:
from flask import Flask, request, jsonify, render_template

def get_web_service_app(inference_fn, is_colab = True) :

  app = Flask(__name__, template_folder = '')
  if is_colab :
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)

  else :
    from flask_cors import CORS
    CORS(app)

  @app.route('/')
  def index():
    return render_template('index.html')

  @app.route('/api', methods = ['post'])
  def api() :
    query_sentence = request.json['sentence']
    output_data = inference_fn(query_sentence)
    response = jsonify(output_data)
    return response
  return app

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
!rm *html